# MCDSS - MESIIA virtual group.    URV
# **TASK 2. Aggregation operators for numerical data**

The goal of this session is to practise with two simple methods of aggregation: WSM (Weighted Sum Model) that is equivalent to weighted average and afterwards the OWA operator (Ordered Weighted Average).
The exercise will also enable you to the use of Python notebooks and with *mcda library* using Python programming tools.

This exercise is part of the evaluation of the course.

**You must write a report on this exercise and upload it at Moodle before the indicated deadline. You must submit a PDF file that includes the results of each task indicated below. In addition to the results, you must add clear and detailed explanations about the results, including the observations and conclusions at each question.**

The decision problem we will consider is the selection of the best restaurant for a certain decision maker. We will consider different restaurants in Tarragona and 6 criteria.

The first thing we should do is installing the MCDA package. To do that, we use the `pip` command. Run the cell and test if everything went correctly. Take a look to the MCDA package documentation to know how to use it properly. [Documentation](https://py-mcda.readthedocs.io/en/stable/index.html) and [Examples](https://py-mcda.readthedocs.io/en/stable/examples.html)

We will use version `0.4.0`.

In [ ]:
!pip install mcda==0.4.0

## Task 1: **Weighted Average or Weighted Sum**

1. The input data of this MCDA methods is: a list of alternatives, a list of criteria and the weights of the criteria. In this model, weights represent the relative importance of each criterion. The sum of the weights must be 1 unit.

In [ ]:
import mcda
import pandas as pd
from mcda.core.performance_table import *
from mcda.core.scales import *
from mcda.plot import *

alternatives = ["Terrat", "RacoAbat", "Dominos", "Ancora", "Frida", "Barhaus"]

criteria = {
    "Food": 0.3,
    "Personnel": 0.3,
    "Atmosphere": 0.1,
    "Category": 0.1,
    "Location": 0.1,
    "Terrace": 0.1
}
criteria

2. Here you have the performance table. Each row corresponds to one of the restaurants (alternatives) and each column contains the utility scores for the set of criteria. The range of the utility is 0..1 and all criteria are maximized.

In [ ]:
performance_table = PerformanceTable(
    [[0.7, 0.9, 0.6, 0.9, 0.0, 0.4],
     [0.5, 0.7, 0.6, 0.7, 0.2, 0.0],
     [0.2, 0.3, 0.1, 0.2, 0.8, 0.0],
     [0.9, 0.5, 0.4, 0.3, 1.0, 0.7],
     [0.8, 0.7, 0.1, 0.5, 0.1, 0.4],
     [0.9, 1.0, 0.3, 0.8, 0.2, 1.0]],
    alternatives = alternatives,
    criteria = criteria.keys())

performance_table.df

We may represent the alternatives and their scores by means of radar projections, which is a very visual representation.

In [ ]:
create_radar_projection(len(criteria), frame='polygon')

fig = Figure(ncols=6, figsize=(20, 20))

for alternative in alternatives:
    values = performance_table.get_alternative_values(alternative)
    ax = fig.create_add_axis(projection = radar_projection_name(len(criteria.values())))
    ax.title = alternative
    ax.add_plot(
        RadarPlot(
            criteria.values(),
            values.data,
            rlimits = [0,1]
        )
    )

fig.draw()

3. Now we want to apply the WSM aggregation. First, we will multiply the utilities and the corresponding weight.

In [ ]:
weighted_table = performance_table.apply_criteria_weights(criteria)

weighted_table.df

4. Secondly, we sum up all the alternatives to obtain an overall score on each one.

In [ ]:
utilitiesWM = weighted_table.sum(axis=1)
utilitiesWM

5. Once we have obtained the weighed table, let's plot it to see visually which alternative is the best according to the specified criteria weights.

In [ ]:
x = [*range(len(alternatives))]

plot = BarPlot(
    x, utilitiesWM, xticks=x,
    xticklabels = alternatives, xticklabels_tilted = True
)
plot.draw()
plot.axis.title = "Alternatives utility score"
plot.axis.xlabel = "Alternatives"
plot.axis.ylabel = "Utility score"
plot.axis.figure.draw()

*According to this plot, which alternative do you think is the best? Why?*

*Notice that in this representation you can discard visually which are the worst alternatives.*

## Task 2: **Add new alternatives**

6. Modify the dataset including two new alternatives with the following characteristics:
- A restaurant that has the worst evaluations in all the criteria except in Service.
- A restaurant that has the worst evaluations in all the criteria except in Location.

Apply again the WSM and study the results achieved for these new restaurants in comparison with the rest of restaurants. Take into account that you must modify also the *alternatives* array.

In [ ]:
# Alternatives names
new_alternatives = []
# Alternatives scores for each criterion
alternatives_values = {
    "Food": [],
    "Personnel": [],
    "Atmosphere": [],
    "Category": [],
    "Location": [],
    "Terrace": []
}

Now, modify the performance table by adding the utilities as indicated. You can also represent them in radar (spider) graphics if you want.

In [ ]:
all_alternatives = alternatives + new_alternatives

performance_table.df = pd.concat([
    performance_table.df,
    pd.DataFrame(alternatives_values, index = new_alternatives)],
    axis = 0)
performance_table.df

Repeat the steps of the first part of the exercise to calculate the Weigthed Average and plot the results.

## Task 3: **arithmetic non-weighted average**

In this part, you will execute a non-weighted **Arithmetic average**. Thus, it is necessary to modify the criteria weights by establishing the same weights for all criteria.

Now, follow the same steps as in part one to calculate the Weigthed Average and plot the results.

## Task 4: **OWA operator**

Now we will study the Ordered Weighted Average, with the same case study.
First we must import the functions about OWA from the package.

In [ ]:
from mcda.mavt.aggregators import *
from mcda.core.scales import *


7. Rember that OWA does not give weights to criteria, so we assume all criteria have the same importance. We have now to define a set of weights that establish the aggregation policy (from a conjunctive to a disjunctive model).
The weights will be applied to the values after reordering them decreasingly.

You have here a set of weights to use as a first example. Can you say which is the aggregation policy?

In [ ]:
weightsOWA =  [0.1, 0.3, 0.6, 0.0, 0.0, 0.0]
weightsOWA

In [ ]:
owa = OWA(weightsOWA)
utilitiesOWA = owa(performance_table)
utilitiesOWA

8. You should  also represent the scores obtained with the bar plot to facilitate the visualization.

9. Now, we will compare different sets of OWA weights. First you must define a disjunctive policy (with a weight=0 for the 2 worst values). Apply it to the restaurants.

10. Second, you must establish a conjunctive policy (simultaneity is required, most of the criteria must be fulfilled). Apply it to the restaurants.

11. Compare the results of both aggregation policies. Compare also them with respect to the arithmetic average.

## Task 5 **Aggregation policies: descriptors of the OWA weights**

There are different measures to characterise the vector of weights, such as: Balance, Divergence, Entropy and Orness.

In this task, we will practise with the examples of sets of weights given in the following table:

| Weights | Balance | Divergence | Entropy| Orness |
| --- | --- | --- | --- | --- |
| 0.0, 0.3, 0.4, 0.3, 0.0, 0.0 | - | - | - | - |
| 0.0, 0.0, 0.0, 0.2, 0.3, 0.5 | - | - | - | - |
| 1.0, 0.0, 0.0, 0.0, 0.0, 0.0 | - | - | - | - |
| 0.1, 0.1, 0.3, 0.3, 0.1, 0.1 | - | - | - | - |
| 0.1, 0.3, 0.6, 0.0, 0.0, 0.0 | - | - | - | - |

12. The MCDA package only includes the orness and dispersion measures, for the moment. So, below you have the functions that implement the characterisation measures of OWA weights. The input of these functions is a vector of weights. So they must be executed for each row of the table.

In [ ]:
examples_OWA_weights = [[0.0, 0.3, 0.4, 0.3, 0.0, 0.0], ...]

for weights in examples_OWA_weights:
    print(weights)
    example_OWA = OWA(weights)

    print(f"Balance: {example_OWA.balance}")
    print(f"Divergence: {example_OWA.divergence}")
    print(f"Entropy: {example_OWA.dispersion}")
    print(f"Orness: {example_OWA.orness}\n")
